In [95]:
import sqlite3
import pandas as pd

In [96]:
DATA_DIR = 'data/'

In [97]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()

# Drop table from previous sessions
cur.execute('DROP TABLE IF EXISTS temperatures')
cur.execute('DROP TABLE IF EXISTS cities')

conn.commit()

In [98]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS cities (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        country TEXT  
    )
''')
conn.commit()

In [99]:
cur.execute('''
                CREATE TABLE IF NOT EXISTS temperatures (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    timestamp TEXT,
                    temperature REAL,
                    city_id INTEGER
                )
            ''')
conn.commit()

In [100]:
cur.execute('''
    INSERT INTO cities (name, country) VALUES ('Uppsala', 'Sweden')
''')
conn.commit()

In [94]:
def insert_temps_from_csv(file_path, city_name):
    data = pd.read_csv(file_path, sep=";", skiprows=10)
    data['DatumTid'] = data['Datum'] + ' ' + data['Tid (UTC)']

    data = data[['DatumTid', 'Lufttemperatur']]

    city_id_resp = cur.execute('SELECT id FROM cities WHERE name = ?', (city_name, )).fetchone()
    if city_id_resp is None:
        print(f"City {city_name} not found")
        resp = cur.execute('''
            INSERT INTO cities (name, country) VALUES (?, ?)
        ''', (city_name, 'Sweden'))
        city_id = resp.lastrowid
    else:
        city_id = city_id_resp[0]

    for row in data.itertuples():
        # print(row[1], row[2])
        cur.execute('''
                        INSERT INTO temperatures (timestamp, temperature, city_id)
                        VALUES (?, ?, ?)
                    ''', (row[1], row[2], city_id))
        # f"""INSERT INTO temperatures (timestamp, temperature) VALUES ('{row[1]}', {row[2]})"""
        # elak_row1 = "'); DROP TABLE temperatures; --"
        # break
    
    conn.commit()

In [101]:
insert_temps_from_csv(DATA_DIR + 'temp-uppsala.csv', 'Uppsala')
insert_temps_from_csv(DATA_DIR + 'temp-lulea.csv', 'Luleå')

/var/folders/pg/6tvtzc815214nstb4nc45dcw0000gn/T/ipykernel_5484/476812076.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=";", skiprows=10)


City Luleå not found


In [111]:
# cur.execute('SELECT * FROM temperatures WHERE temperature > 30')
# user = cur.fetchone()

for row in cur.execute("""
                        SELECT MAX(temperatures.temperature), MIN(temperatures.temperature), cities.name FROM temperatures
                        JOIN cities ON temperatures.city_id = cities.id
                        GROUP BY cities.name
                        LIMIT 10
                       """):
    print(row)

(34.0, -39.7, 'Luleå')
(33.3, -27.6, 'Uppsala')


In [65]:
# conn.close()